In [1]:
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.io as pio
import ta
import pandas as pd
import numpy as np
import yfinance as yf
from pandas import DataFrame
# Set the default template to plotly_dark
pio.templates.default = 'plotly_dark'

In [7]:
import yfinance as yf
import pandas as pd


class StockData:
    """
    A class that retrieves historical stock data from Yahoo Finance using the yfinance library.

    Attributes:
        symbol (str): The stock symbol to retrieve data for.

        start_date (str): The start date of the date range to retrieve data for, in YYYY-MM-DD format.

        end_date (str): The end date of the date range to retrieve data for, in YYYY-MM-DD format.

        interval (str): The time interval between data points. Valid values are '1d' (daily), '1wk' (weekly), '1mo' (monthly), '1m' (1-minute), '2m' (2-minute), '5m' (5-minute), '15m' (15-minute), '30m' (30-minute), '60m' (60-minute), '90m' (90-minute), and '1h' (hourly).

        data (pandas.DataFrame): The retrieved stock data.
    """

    def __init__(self, symbol, start_date=None, end_date=None, interval="1d") -> None:

        self.startup(symbol, start_date, end_date, interval)

    def startup(self, symbol, start_date, end_date, interval) -> None:
        """
        Constructs a new StockData object and downloads the stock data from Yahoo Finance.

        Args:
            symbol (str): The stock symbol to retrieve data for.

            start_date (str): The start date of the date range to retrieve data for, in YYYY-MM-DD format.

            end_date (str): The end date of the date range to retrieve data for, in YYYY-MM-DD format.

            interval (str, optional): The time interval between data points. Default is '1d' for daily data. Valid values are '1d' (daily), '1wk' (weekly), '1mo' (monthly), '1m' (1-minute), '2m' (2-minute), '5m' (5-minute), '15m' (15-minute), '30m' (30-minute), '60m' (60-minute), '90m' (90-minute), and '1h' (hourly).
        """
        self.symbol: str = symbol
        self.start_date: str = start_date
        self.end_date: str = end_date
        self.interval: str = interval
        self.data: DataFrame = yf.download(
            symbol, start=start_date, end=end_date, interval=interval
        )

    def __repr__(self) -> str:
        """
        Returns a string representation of the StockData object.
        """
        return f"StockData(symbol={self.symbol}, start_date={self.start_date}, end_date={self.end_date}, interval={self.interval})"

    def __str__(self) -> str:
        """
        Returns a string representation of the stock data.
        """
        return str(self.data)

In [10]:
stockdata = StockData(
    symbol='TSLA'
)
df: DataFrame = stockdata.data

[*********************100%***********************]  1 of 1 completed


In [22]:
class TechnicalIndicators:
    """
    A class that calculates various technical indicators for a given stock price dataset.
    """

    def __init__(
        self, data, sma_window=20, ema_window=20, rsi_window=14, adx_window=14
    ):
        """
        Constructs a new TechnicalIndicators object.

        Args:
            data (pandas.DataFrame): The stock price dataset to calculate technical indicators for.
            sma_window (int, optional): The window size for the simple moving average. Default is 20.
            ema_window (int, optional): The window size for the exponential moving average. Default is 20.
            rsi_window (int, optional): The window size for the Relative Strength Index. Default is 14.
            adx_window (int, optional): The window size for the Average Directional Index. Default is 14.
        """
        self.data = data
        self.sma_window = sma_window
        self.ema_window = ema_window
        self.rsi_window = rsi_window
        self.adx_window = adx_window

    def calculate_sma(self):
        """
        Calculates the simple moving average and adds it to the dataset.

        Returns:
            pandas.Series: The simple moving average.
        """
        sma = ta.trend.sma_indicator(self.data["Close"], window=self.sma_window)
        self.data["SMA"] = sma
        return sma

    def calculate_ema(self):
        """
        Calculates the exponential moving average and adds it to the dataset.

        Returns:
            pandas.Series: The exponential moving average.
        """
        ema = ta.trend.ema_indicator(self.data["Close"], window=self.ema_window)
        self.data["EMA"] = ema
        return ema

    def calculate_macd(self):
        """
        Calculates the Moving Average Convergence Divergence (MACD) and adds it to the dataset.

        Returns:
            pandas.DataFrame: A dataframe with the MACD, signal line, and histogram.
        """
        macd, signal, hist = (
            ta.trend.macd(self.data["Close"]),
            ta.trend.macd_signal(self.data["Close"]),
            ta.trend.macd_diff(self.data["Close"]),
        )
        self.data["MACD"] = macd
        self.data["Signal"] = signal
        self.data["Histogram"] = hist
        return self.data[["MACD", "Signal", "Histogram"]]

    def calculate_rsi(self):
        """
        Calculates the Relative Strength Index (RSI) and adds it to the dataset.

        Returns:
            pandas.Series: The RSI.
        """
        rsi = ta.momentum.rsi(self.data["Close"], window=self.rsi_window)
        self.data["RSI"] = rsi
        return rsi

    def calculate_adx(self):
        """
        Calculates the Average Directional Index (ADX) and adds it to the dataset.

        Returns:
            pandas.Series: The ADX.
        """
        adx = ta.trend.adx(
            self.data["High"],
            self.data["Low"],
            self.data["Close"],
            window=self.adx_window,
        )
        self.data["ADX"] = adx
        return adx

    def calculate_obv(self):
        """
        Calculates the On-Balance Volume (OBV) and adds it to the dataset.

        Returns:
            pandas.Series: The OBV.
        """
        obv = ta.volume.on_balance_volume(self.data["Close"], self.data["Volume"])
        self.data["OBV"] = obv
        return obv

    def calculate_all(
        self, sma_window=None, ema_window=None, rsi_window=None, adx_window=None
    ):
        """
        Calculates all technical indicators and adds them to the dataset.

        Args:
            sma_window (int, optional): The window size for the simple moving average. Default is the value specified in the constructor.
            ema_window (int, optional): The window size for the exponential moving average. Default is the value specified in the constructor.
            rsi_window (int, optional): The window size for the Relative Strength Index. Default is the value specified in the constructor.
            adx_window (int, optional): The window size for the Average Directional Index. Default is the value specified in the constructor.

        Returns:
            pandas.DataFrame: The complete dataset with the technical indicators appended to it.
        """
        if sma_window is not None:
            self.sma_window = sma_window
        if ema_window is not None:
            self.ema_window = ema_window
        if rsi_window is not None:
            self.rsi_window = rsi_window
        if adx_window is not None:
            self.adx_window = adx_window

        data_copy = self.data.copy()

        self.calculate_sma()
        self.calculate_ema()
        self.calculate_macd()
        self.calculate_rsi()
        self.calculate_adx()
        self.calculate_obv()

        return self.data


In [23]:
df_full: DataFrame = TechnicalIndicators(df).calculate_all()

c:\Users\JustAnotherBot\miniconda3\envs\ibmenv\lib\site-packages\ta\trend.py:721: FutureWarning:

The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.

c:\Users\JustAnotherBot\miniconda3\envs\ibmenv\lib\site-packages\ta\trend.py:769: RuntimeWarning:

invalid value encountered in scalar divide

c:\Users\JustAnotherBot\miniconda3\envs\ibmenv\lib\site-packages\ta\trend.py:774: RuntimeWarning:

invalid value encountered in scalar divide

